# 資優生策略

In [3]:
import pandas as pd

def toSeasonal(df):
    season4 = df[df.index.month == 3]
    season1 = df[df.index.month == 5]
    season2 = df[df.index.month == 8]
    season3 = df[df.index.month == 11]

    season1.index = season1.index.year
    season2.index = season2.index.year
    season3.index = season3.index.year
    season4.index = season4.index.year - 1

    newseason1 = season1
    newseason2 = season2 - season1.reindex_like(season2)
    newseason3 = season3 - season2.reindex_like(season3)
    newseason4 = season4 - season3.reindex_like(season4)

    newseason1.index = pd.to_datetime(newseason1.index.astype(str) + '-05-15')
    newseason2.index = pd.to_datetime(newseason2.index.astype(str) + '-08-14')
    newseason3.index = pd.to_datetime(newseason3.index.astype(str) + '-11-14')
    newseason4.index = pd.to_datetime((newseason4.index + 1).astype(str) + '-03-31')

    return newseason1.append(newseason2).append(newseason3).append(newseason4).sort_index()

def mystrategy2(data):
    
    股本 = data.get('股本合計', 1)#.drop_duplicates(['stock_id', 'date'], keep='last')#.pivot(index='date', columns='stock_id')
    price = data.get('收盤價', 200)
    當天股價 = price[:股本.index[-1]].iloc[-1]
    當天股本 = 股本.iloc[-1]
    市值 = 當天股本 * 當天股價 / 10 * 1000
   

    df1 = toSeasonal(data.get('投資活動之淨現金流入（流出）', 5))
    df2 = toSeasonal(data.get('營業活動之淨現金流入（流出）', 5))
    自由現金流 = (df1 + df2).iloc[-4:].mean()
    
    
    稅後淨利 = data.get('本期淨利（淨損）', 1)
    
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 1)
    權益總額 = data.get('權益總額', 1)
    
    # 並且把它們合併起來
    權益總計.fillna(權益總額, inplace=True)
        
    股東權益報酬率 = 稅後淨利.iloc[-1] / 權益總計.iloc[-1]
    
    
    營業利益 = data.get('營業利益（損失）', 5)
    營業利益成長率 = (營業利益.iloc[-1] / 營業利益.iloc[-5] - 1) * 100
    
    
    當月營收 = data.get('當月營收', 4) * 1000
    當季營收 = 當月營收.iloc[-4:].sum()
    市值營收比 = 市值 / 當季營收
    
    rsv = (price.iloc[-1] - price.iloc[-150:].min()) / (price.iloc[-150:].max() - price.iloc[-150:].min())
    
    
    condition1 = (市值 < 1e10)
    condition2 = 自由現金流 > 0
    condition3 = 股東權益報酬率 > 0
    condition4 = 營業利益成長率 > 0
    condition5 = 市值營收比 < 5
    condition6 = rsv > 0.5
    
    select_stock = condition1 & condition2 & condition3 & condition4 & condition5 & condition6
    
    return select_stock[select_stock]


In [92]:
def strategy_Buffett(data):
    
    #此策略僅兩個條件:  1.近三年平均ROE>.15  2.最近年度股東權益報酬率＞平均值(市場及產業)
    稅後淨利 = data.get('本期淨利（淨損）', 10)
    
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 10)
    權益總額 = data.get('權益總額', 10)
    
    # 並且把它們合併起來    
    權益總計.fillna(權益總額, inplace=True)
    
    #條件1.ROE近三年平均
    近三年平均ROE = ((稅後淨利.iloc[-1] / 權益總計.iloc[-1])+(稅後淨利.iloc[-5] / 權益總計.iloc[-5])+(稅後淨利.iloc[-9] / 權益總計.iloc[-9]))/3

    condition1 = 近三年平均ROE > 0.15
    
    #條件2.最近年度股東權益報酬率＞平均值(市場及產業)

    #找出所有股
    allstocklist = []
    稅後淨利 = data.get('本期淨利（淨損）', 1)
    for i in 稅後淨利.keys():
        allstocklist.append(i)
        
    #找出同類股
    stockindex = []
    for i in range(len(condition1[condition1])):
        stockindex.append(int(condition1[condition1].index[i]))

    from collections import defaultdict
    samekind = defaultdict(list)

    for j in range(len(stockindex)):
        for i in range(len(allstocklist)):
            if(str(int(allstocklist[i])//100)==str(int(stockindex[j])//100)):
                samekind[stockindex[j]].append(allstocklist[i])
    
    totalni = defaultdict(list)

    for i in samekind.keys():
        sumroe = 0
        avgroe = 0
        for j in samekind[i]:
            sumroe = (稅後淨利[j].iloc[-1]/權益總計[j].iloc[-1])+sumroe
        avgroe = sumroe/len(samekind[i])
        totalni[i].append(avgroe)
    
    final = []
    
    for i in range(len(stockindex)):
        if (稅後淨利[stockindex[i]].iloc[-1]/權益總計[stockindex[i]].iloc[-1])>totalni[stockindex[i]]:
                final.append(stockindex[i])
                print(final)

    #condition2 = condition1[condition1]>totalni[condition1[condition1]]

    #condition = condition1 & condition2

    #return condition1[condition1]


from finlab.data import Data
data = Data()
results = strategy_Buffett(data)
results
   

KeyError: 1475

In [89]:
s = ["1475","2330"]
稅後淨利 = data.get('本期淨利（淨損）', 10)
    
# 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
# 所以得把它們抓出來
權益總計 = data.get('權益總計', 10)
權益總額 = data.get('權益總額', 10)
    
# 並且把它們合併起來    
權益總計.fillna(權益總額, inplace=True)
final = []
for i in range(len(s)):
        if (稅後淨利[s[i]].iloc[-1]/權益總計[s[i]].iloc[-1])>0.02:
                final.append(s[i])
print(final)

['2330']


In [79]:
type(results)
results.index[0]
len(results)

stockindex = []
for i in range(len(results)):
    stockindex.append(results.index[i])
    
type(stockindex[0])

str

In [121]:
def strategy_Buffett(data):
    
    #此策略僅兩個條件:  1.近三年平均ROE>.15  2.最近年度股東權益報酬率＞平均值(市場及產業)
    稅後淨利 = data.get('本期淨利（淨損）', 10)
    
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 10)
    權益總額 = data.get('權益總額', 10)
    
    # 並且把它們合併起來    
    權益總計.fillna(權益總額, inplace=True)
    
    #條件1.ROE近三年平均
    近三年平均ROE = ((稅後淨利.iloc[-1] / 權益總計.iloc[-1])+(稅後淨利.iloc[-5] / 權益總計.iloc[-5])+(稅後淨利.iloc[-9] / 權益總計.iloc[-9]))/3

    condition1 = 近三年平均ROE > 0.15
    
    results = condition1[condition1]
    
    allstocklist = []
    稅後淨利 = data.get('本期淨利（淨損）', 1)
    for i in 稅後淨利.keys():
        allstocklist.append(i)
        
    #找出同類股
    stockindex = []
    for i in range(len(results)):
        stockindex.append(int(results.index[i]))

    from collections import defaultdict
    samekind = defaultdict(list)

    for j in range(len(stockindex)):
        for i in range(len(allstocklist)):
            if(str(int(allstocklist[i])//100)==str(int(stockindex[j])//100)):
                samekind[stockindex[j]].append(allstocklist[i])
    
    totalni = defaultdict(list)

    for i in samekind.keys():
        sumroe = 0
        avgroe = 0
        for j in samekind[i]:
            sumroe = (稅後淨利[j].iloc[-1]/權益總計[j].iloc[-1])+sumroe
        avgroe = sumroe/len(samekind[i])
        totalni[i].append(avgroe)
    
    print(totalni)
    
    final = []
    
    for i in range(len(stockindex)):
        if (稅後淨利[str(stockindex[i])].iloc[-1]/權益總計[str(stockindex[i])].iloc[-1])>totalni[stockindex[i]]:
            final.append(stockindex[i])
    return final       

from finlab.data import Data
data = Data()
results = strategy_Buffett(data)
results

defaultdict(<class 'list'>, {1475: [0.005165974137401159], 3043: [0.01381072548092402]})


[3043]

In [68]:
stockindex 

['1475', '3043']

In [40]:
#近三年平均ROE>0.15
def condition1(data):
    # 爬取稅後淨利
    稅後淨利 = data.get('本期淨利（淨損）', 10)
    
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 10)
    權益總額 = data.get('權益總額', 10)
    
    # 並且把它們合併起來    
    權益總計.fillna(權益總額, inplace=True)
    
    #條件1.ROE近三年平均
    近三年平均ROE = ((稅後淨利.iloc[-1] / 權益總計.iloc[-1])+(稅後淨利.iloc[-5] / 權益總計.iloc[-5])+(稅後淨利.iloc[-9] / 權益總計.iloc[-9]))/3

    condition1 = 近三年平均ROE > 0.15
    return condition1[condition1]

In [117]:
def condition2(data):
    # 爬取稅後淨利
    稅後淨利 = data.get('本期淨利（淨損）', 10)
    
    # 股東權益，有兩個名稱，有些公司叫做權益總計，有些叫做權益總額
    # 所以得把它們抓出來
    權益總計 = data.get('權益總計', 10)
    權益總額 = data.get('權益總額', 10)
    
    # 並且把它們合併起來    
    權益總計.fillna(權益總額, inplace=True)
    
    allstocklist = []
    稅後淨利 = data.get('本期淨利（淨損）', 1)
    for i in 稅後淨利.keys():
        allstocklist.append(i)
        
    #找出同類股
    stockindex = []
    for i in range(len(data)):
        stockindex.append(int(data.index[i]))

    from collections import defaultdict
    samekind = defaultdict(list)

    for j in range(len(stockindex)):
        for i in range(len(allstocklist)):
            if(str(int(allstocklist[i])//100)==str(int(stockindex[j])//100)):
                samekind[stockindex[j]].append(allstocklist[i])
    
    totalni = defaultdict(list)

    for i in samekind.keys():
        sumroe = 0
        avgroe = 0
        for j in samekind[i]:
            sumroe = (稅後淨利[j].iloc[-1]/權益總計[j].iloc[-1])+sumroe
        avgroe = sumroe/len(samekind[i])
        totalni[i].append(avgroe)
    
    print(totalni)
    
    final = []
    
    for i in range(len(stockindex)):
        if (稅後淨利[str(stockindex[i])].iloc[-1]/權益總計[str(stockindex[i])].iloc[-1])>totalni[stockindex[i]]:
            final.append(stockindex[i])
    return final       

In [119]:
from finlab.data import Data
data = Data()
results = condition1(data)
results

stock_id
1475    True
3043    True
dtype: bool

In [120]:
a = condition2(results)
a

AttributeError: 'int' object has no attribute 'fillna'

In [116]:
allstocklist = []
稅後淨利 = data.get('本期淨利（淨損）', 1)
for i in 稅後淨利.keys():
    allstocklist.append(i)
        
#找出同類股
stockindex = []
for i in range(len(aaa)):
    stockindex.append(int(aaa.index[i]))

from collections import defaultdict
samekind = defaultdict(list)

for j in range(len(stockindex)):
    for i in range(len(allstocklist)):
        if(str(int(allstocklist[i])//100)==str(int(stockindex[j])//100)):
            samekind[stockindex[j]].append(allstocklist[i])
    
totalni = defaultdict(list)

for i in samekind.keys():
    sumroe = 0
    avgroe = 0
    for j in samekind[i]:
        sumroe = (稅後淨利[j].iloc[-1]/權益總計[j].iloc[-1])+sumroe
    avgroe = sumroe/len(samekind[i])
    totalni[i].append(avgroe)
    
print(totalni)
    
final = []
    
for i in range(len(stockindex)):
    if (稅後淨利[str(stockindex[i])].iloc[-1]/權益總計[str(stockindex[i])].iloc[-1])>totalni[stockindex[i]]:
            final.append(stockindex[i])
            print(final)

defaultdict(<class 'list'>, {1475: [0.005165974137401159], 3043: [0.01381072548092402]})
[3043]


In [115]:
稅後淨利[str(stockindex[0])].iloc[-1]

269.0

In [97]:
stockindex[0]

1475

In [98]:
totalni[stockindex[0]]

[0.005165974137401159]

In [49]:
from finlab.data import Data
data = Data()
results = condition1(data)
results

stock_id
1475    True
3043    True
dtype: bool

In [6]:
#取得所有股票代碼
allstocklist = []
稅後淨利 = data.get('本期淨利（淨損）', 1)
for i in 稅後淨利.keys():
    allstocklist.append(i)

allstocklist


['1101',
 '1102',
 '1103',
 '1104',
 '1108',
 '1109',
 '1110',
 '1201',
 '1203',
 '1210',
 '1213',
 '1215',
 '1216',
 '1217',
 '1218',
 '1219',
 '1220',
 '1225',
 '1227',
 '1229',
 '1231',
 '1232',
 '1233',
 '1234',
 '1235',
 '1236',
 '1240',
 '1256',
 '1258',
 '1259',
 '1264',
 '1268',
 '1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1319',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1336',
 '1337',
 '1338',
 '1339',
 '1340',
 '1341',
 '1342',
 '1402',
 '1409',
 '1410',
 '1413',
 '1414',
 '1416',
 '1417',
 '1418',
 '1419',
 '1423',
 '1432',
 '1434',
 '1435',
 '1436',
 '1437',
 '1438',
 '1439',
 '1440',
 '1441',
 '1442',
 '1443',
 '1444',
 '1445',
 '1446',
 '1447',
 '1449',
 '1451',
 '1452',
 '1453',
 '1454',
 '1455',
 '1456',
 '1457',
 '1459',
 '1460',
 '1463',
 '1464',
 '1465',
 '1466',
 '1467',
 '1468',
 '1470',
 '1471',
 '1472',
 '1473',
 '1474',
 '1475',
 '1476',
 '1477',
 '1503',
 '1504',
 '1506',
 '1507',
 

In [30]:
#找出同類股
stockindex = list(results.keys)
from collections import defaultdict
samekind = defaultdict(list)

for j in range(len(stockindex)):
    for i in range(len(allstocklist)):
        if(str(int(allstocklist[i])//100)==str(int(stockindex[j])//100)):
            samekind[stockindex[j]].append(allstocklist[i])
            
samekind

defaultdict(list,
            {'1101': ['1101', '1102', '1103', '1104', '1108', '1109', '1110'],
             '2330': ['2301',
              '2302',
              '2303',
              '2305',
              '2308',
              '2312',
              '2313',
              '2314',
              '2316',
              '2317',
              '2321',
              '2323',
              '2324',
              '2327',
              '2328',
              '2329',
              '2330',
              '2331',
              '2332',
              '2337',
              '2338',
              '2340',
              '2342',
              '2344',
              '2345',
              '2347',
              '2348',
              '2349',
              '2351',
              '2352',
              '2353',
              '2354',
              '2355',
              '2356',
              '2357',
              '2358',
              '2359',
              '2360',
              '2362',
              '2363',
              '

In [36]:
稅後淨利 = data.get('本期淨利（淨損）', 1)
權益總計 = data.get('權益總計', 1)
權益總額 = data.get('權益總額', 1)
權益總計.fillna(權益總額, inplace=True)

totalni = defaultdict(list)

for i in samekind.keys():
    sumroe = 0
    avgroe = 0
    for j in samekind[i]:
        sumroe = (稅後淨利[j].iloc[-1]/權益總計[j].iloc[-1])+sumroe
    avgroe = sumroe/len(samekind[i])
    totalni[i].append(avgroe)  
    
print(totalni)

defaultdict(<class 'list'>, {'1101': [0.02791638689628332], '2330': [0.03213144707874767]})


In [33]:
totalni[]

IndexError: list index out of range

# 檢查報酬率

In [ ]:
# 起手式
from finlab.backtest import backtest
from finlab.data import Data
import datetime
%matplotlib inline

# 回測
data = Data()
backtest(datetime.date(2018,3,1), datetime.date(2018,3,10), 60, mystrategy2, data)

# 更新持股投資張數

In [ ]:
from finlab.backtest import portfolio

data = Data()

# 利用 mystrategy2 來產生股票清單 stocks
stocks = mystrategy2(data)

# 用portfolio來幫忙計算，給定 1000000 元，依照今天收盤價，股票張數要如何分配
p, total_invest_money = portfolio(stocks.index, 1000000, data)

# 印出股票資訊
print('---------------')
print('|  portfolio  |')
print('---------------')
print(p)
print('total cost')
print(total_invest_money)